<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/yfinance_bist_bilanco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gerekli kütüphaneler
!pip install investpy xlsxwriter

import investpy
import yfinance as yf
import pandas as pd
import xlsxwriter
from requests.exceptions import HTTPError

# Tüm BIST hisselerini çekelim
turkish_stocks = investpy.stocks.get_stocks(country='Turkey')
hisseler = turkish_stocks['symbol'].tolist()
hisse_listesi = [hisse + ".IS" for hisse in hisseler]

veriler = []
hatali_hisseler = []

print("Tarama başladı... devam ediyor")
for hisse in hisse_listesi:
    try:
        ticker = yf.Ticker(hisse)
        finansal_veri = ticker.info
        balance_sheet = ticker.balance_sheet
        income_statement = ticker.financials
        cashflow = ticker.cashflow

        if not finansal_veri or "quoteType" not in finansal_veri:
            continue
        if balance_sheet.empty:
            continue
        if income_statement.empty:
            continue
        if cashflow.empty:
            continue

        # Finansal metrikler
        firma_degeri = finansal_veri.get("enterpriseValue", 0)
        piyasa_degeri = finansal_veri.get("marketCap", 0)
        nakit = finansal_veri.get("totalCash", 0)
        defter_degeri = finansal_veri.get("bookValue", 0)
        ebitda = income_statement.loc['EBITDA'].iloc[0] if 'EBITDA' in income_statement.index else None
        net_kar = finansal_veri.get("netIncomeToCommon", 0)
        toplam_gelir = income_statement.loc['Total Revenue'].iloc[0] if 'Total Revenue' in income_statement.index else None
        toplam_borc = balance_sheet.loc['Total Debt'].iloc[0] if 'Total Debt' in balance_sheet.index else None
        hisse_basi_gelir = finansal_veri.get("revenuePerShare", 0)
        son_ceyrek_kar = finansal_veri.get("earningsQuarterlyGrowth", 0)
        brut_kar_marji = finansal_veri.get("grossMargins", 0)
        brut_kar = income_statement.loc['Gross Profit'].iloc[0] if 'Gross Profit' in income_statement.index else None
        faaliyet_kar_marji = finansal_veri.get("operatingMargins", 0)
        varlik_karliligi = finansal_veri.get("returnOnAssets", 0)
        ozsermaye_karliligi = finansal_veri.get("returnOnEquity", 0)
        fk_trailing = finansal_veri.get("trailingPE", 0)
        fk_forward = finansal_veri.get("forwardPE", 0)
        hizli_oran = finansal_veri.get("quickRatio", 0)
        borc_ozkaynak_orani = finansal_veri.get("debtToEquity", 0)
        cari_oran = finansal_veri.get("currentRatio", 0)
        #net_borc = toplam_borc - nakit
        pd_dd = ticker.info.get('marketCap') / ticker.info.get('bookValue')
        peg_orani = finansal_veri.get("pegRatio", 0)
        operating_cashflow = finansal_veri.get("operatingCashflow", None)
        sermaye_harcamalari = finansal_veri.get("capitalExpenditures", None)
        serbestnakitakisi = finansal_veri.get("freeCashflow", None)
        ozkaynak = finansal_veri.get("totalStockholderEquity", None)
        uzunvadeliborc = finansal_veri.get("longTermDebt", None)
        kisavadeborc = finansal_veri.get("totalCurrentLiabilities", None)
        fd_favok = ticker.info['enterpriseValue'] / ticker.info['ebitda'] if ticker.info['ebitda'] else None
        #fd_favok = ticker.info.get('enterpriseValue', None) / ticker.info.get('ebitda', None) if ticker.info.get('ebitda', None) else None
        toplam_varliklar = balance_sheet.loc['Total Assets'].iloc[0] if 'Total Assets' in balance_sheet.index else None
        donen_varliklar = balance_sheet.loc['Current Assets'].iloc[0] if 'Current Assets' in balance_sheet.index else None
        toplam_yukumlulukler = balance_sheet.loc['Total Liabilities Net Minority Interest'].iloc[0] if 'Total Liabilities Net Minority Interest' in balance_sheet.index else None
        donen_yukumlulukler = balance_sheet.loc['Current Liabilities'].iloc[0] if 'Current Liabilities' in balance_sheet.index else None
        duran_yukumlulukler = balance_sheet.loc['Total Non Current Liabilities Net Minority Interest'].iloc[0] if 'Total Non Current Liabilities Net Minority Interest' in balance_sheet.index else None
        azinlik_hakki = balance_sheet.loc['Minority Interest'].iloc[0] if 'Minority Interest' in balance_sheet.index else None
        net_borc = balance_sheet.loc['Net Debt'].iloc[0] if 'Net Debt' in balance_sheet.index else None
        faaliyet_gideri = income_statement.loc['Operating Expense'].iloc[0] if 'Operating Expense' in income_statement.index else None
        faaliyet_geliri = income_statement.loc['Operating Income'].iloc[0] if 'Operating Income' in income_statement.index else None
        net_gelir = income_statement.loc['Net Income'].iloc[0] if 'Net Income' in income_statement.index else None
        faiz_gideri = income_statement.loc['Interest Expense'].iloc[0] if 'Interest Expense' in income_statement.index else None
        vergi_oncesi_kar = income_statement.loc['Pretax Income'].iloc[0] if 'Pretax Income' in income_statement.index else None
        hisse_basina_kar = income_statement.loc['Diluted EPS'].iloc[0] if 'Diluted EPS' in income_statement.index else None


        # Net Kar Marjı (%) hesapla
        if toplam_gelir != 0:
            net_kar_marji = (net_kar / toplam_gelir) * 100
        else:
            net_kar_marji = 0

        veriler.append({
            "Hisse": hisse,
            "Sektör": finansal_veri.get("sector", "N/A"),
            "Analist hedef fiyat": finansal_veri.get("targetMeanPrice", 0),
            "Firma Değeri (₺)": firma_degeri,
            "Dolaşımdaki Hisse Adedi": finansal_veri.get("sharesOutstanding", 0),
            "Piyasa Değeri (₺)": piyasa_degeri,
            "Defter Değeri (₺)": defter_degeri,
            "PD/DD": pd_dd,
            "FAVÖK (₺)": ebitda,
            "FD/FAVÖK": fd_favok,
            "EBITDA (₺)": ebitda,
            "Çeyrek Kâr Büyümesi (%)": son_ceyrek_kar,
            "Kâr Büyümesi (%)": finansal_veri.get("earningsGrowth", 0),
            "Gelir Büyümesi (%)": finansal_veri.get("revenueGrowth", 0),
            "Varlık Karlılığı (ROA) (%)": varlik_karliligi,
            "Özsermaye Karlılığı (ROE) (%)": ozsermaye_karliligi,
            "Net Kar Marjı (%)": net_kar_marji,
            "Borç/Özkaynak Oranı": borc_ozkaynak_orani,
            "FK (Trailing PE)": fk_trailing,
            "FK (Forward PE)": fk_forward,
            "Nakit (₺)": nakit,
            "Net Borç (₺)": net_borc,
            "Toplam Borç (₺)": toplam_borc,
            "Hisse Başına Gelir (₺)": hisse_basi_gelir,
            "Brüt Kar Marjı": brut_kar_marji,
            "Toplam Gelir": toplam_gelir,
            "Faaliyet Geliri (₺)": faaliyet_geliri,
            "Faaliyet Kar Marjı (%)": faaliyet_kar_marji,
            "Faaliyet Gideri (₺)": faaliyet_gideri,
            "Toplam Varlıklar (₺)": toplam_varliklar,
            "Dönen Varlıklar (₺)": donen_varliklar,
            "Toplam Yükümlülükler (₺)": toplam_yukumlulukler,
            "Dönen Yükümlülükler (₺)": donen_yukumlulukler,
            "Duran Yükümlülükler (₺)": duran_yukumlulukler,
            "Kısa Vadeli Yükümlülükler (₺)": kisavadeborc,
            "Uzun Vadeli Borçlar (₺)": uzunvadeliborc,
            "Özkaynak (₺)": ozkaynak,
            "Azınlık Hakkı (₺)": azinlik_hakki,
            "Net Borç (₺)": net_borc,
            "Net Gelir (₺)": net_gelir,
            "Faiz Gideri (₺)": faiz_gideri,
            "Vergi Öncesi Kâr (₺)": vergi_oncesi_kar,
            "Brüt Kâr (₺)": brut_kar,
            "Net Kâr (₺)": net_kar,
            "Seyreltilmiş Hisse Başına Kâr (₺)": hisse_basina_kar,
            "İşletme Faaliyetlerinden Gelen Nakit Akışı (₺)": operating_cashflow,
            "Sermaye Harcamaları (₺)": sermaye_harcamalari,
            "Serbest Nakit Akışı (₺)": serbestnakitakisi,
            "Cari Oran": cari_oran,
            "Hızlı Oran": hizli_oran,
            "Temettü Getirisi (%)": finansal_veri.get("dividendYield", 0),
            "Temettü Oranı": finansal_veri.get("dividendRate", 0),
        })
    except HTTPError as http_err:
        if "404" not in str(http_err):
            hatali_hisseler.append(hisse)
    except Exception as e:
        hatali_hisseler.append(hisse)

# DataFrame
print("Tarama tamamlandı. Veriler sıralanıyor...")
df = pd.DataFrame(veriler)
df = df.sort_values(by="Net Kâr (₺)", ascending=False)


# Excel
from datetime import datetime
tarih = datetime.now().strftime("%Y-%m-%d")
dosya_adi = f"Finansal_Veriler_{tarih}.xlsx"

with pd.ExcelWriter(dosya_adi, engine='xlsxwriter') as writer:
    df['Hisse'] = df['Hisse'].str.replace('.IS', '', regex=False)
    df.to_excel(writer, index=False, sheet_name='Finansal Veriler')
    workbook = writer.book
    worksheet = writer.sheets['Finansal Veriler']
    worksheet.set_zoom(85)

    number_format = workbook.add_format({'num_format': '#,##0.00'})
    for idx, col in enumerate(df.columns):
        if df[col].dtype in ['float64', 'int64']:
            worksheet.set_column(idx, idx, 18, number_format)

# Anlık Görüntüleme
print("Finansal veriler başarıyla kaydedildi ve sıralandı.")
display(df)

print(f"Tarama bitti. Başarısız hisseler: {len(hatali_hisseler)}")

Tarama başladı... devam ediyor


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/EGCYH.IS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=EGCYH.IS&crumb=wkO00cqluYH
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/EGCYO.IS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=EGCYO.IS&crumb=wkO00cqluYH
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ITTFH.IS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ITTFH.IS&crumb=wkO00cqluYH
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/IZOCM.IS?modules=financialData%2

Tarama tamamlandı. Veriler sıralanıyor...
Finansal veriler başarıyla kaydedildi ve sıralandı.


,Hisse,Sektör,Analist hedef fiyat,Firma Değeri (₺),Dolaşımdaki Hisse Adedi,Piyasa Değeri (₺),Defter Değeri (₺),PD/DD,FAVÖK (₺),FD/FAVÖK,...,Brüt Kâr (₺),Net Kâr (₺),Seyreltilmiş Hisse Başına Kâr (₺),İşletme Faaliyetlerinden Gelen Nakit Akışı (₺),Sermaye Harcamaları (₺),Serbest Nakit Akışı (₺),Cari Oran,Hızlı Oran,Temettü Getirisi (%),Temettü Oranı
205,THYAO,Industrials,430.99230,669917511680,1375200000,389525405696,448.266,8.689604e+08,5.202981e+09,7.089899,...,4.044931e+09,182365995008,3.999688,1.901460e+11,None,9.029338e+10,0.974,0.772,0.0000,0.00
215,TUPRS,Energy,193.63818,219692105728,1926800000,274376310784,125.446,2.187207e+09,7.232842e+10,5.673176,...,1.097324e+11,43746828288,256.970000,3.403510e+08,None,8.045274e+09,1.211,0.757,0.1682,23.87
104,FROTO,Consumer Cyclical,1434.55640,429059833856,350910016,328802697216,288.900,1.138119e+09,5.913493e+10,23.077234,...,5.524791e+10,33478248448,139.800000,1.440632e+10,None,-7.170318e+10,1.208,0.737,0.0304,28.50
191,SASA,Consumer Cyclical,0.00000,269996015616,43815600128,177891328000,2.518,7.064787e+10,8.459602e+09,32.033398,...,8.705348e+09,26691508224,0.546538,6.432782e+09,None,-2.315639e+10,0.472,0.211,0.0000,0.00
203,TCELL,Communication Services,137.72200,219272183808,2182109952,203590860800,80.028,2.543995e+09,4.519111e+10,14.120527,...,2.269815e+10,26066542592,5.750000,4.887698e+10,None,1.075890e+10,1.820,1.730,0.0307,2.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,ARCLK,Consumer Cyclical,226.56084,198756057088,606851968,86537093120,124.052,6.975872e+08,2.433216e+10,21.331500,...,7.537799e+10,-1858040960,12.635000,-2.991119e+10,None,-3.019294e+10,1.126,0.686,0.0181,2.66
114,GSDHO,Industrials,0.00000,3858187520,900000000,3510000128,6.543,5.364512e+08,-1.438443e+09,2.356655,...,9.354480e+08,-1965437056,-1.866000,4.087300e+08,None,-2.208408e+09,2.985,2.920,0.0275,0.10
36,BAGFS,Basic Materials,0.00000,7000641024,135000000,2972700160,37.466,7.934394e+07,-1.537683e+09,-7.871821,...,-7.111352e+08,-2011992064,NaN,-1.470112e+09,None,-2.060381e+09,0.628,0.075,0.0055,0.13
111,GOZDE,Financial Services,0.00000,13504504832,385000000,9956100096,68.191,1.460031e+08,-7.855927e+09,3.724757,...,8.345456e+07,-3722516480,-22.870000,1.432431e+08,None,-3.815991e+08,15.050,15.050,0.0000,0.00


Tarama bitti. Başarısız hisseler: 73
